# GFM Affected Population Calculation

This notebook demonstrates the complete GFM (Global Flood Monitoring) affected population methodology using real data from the September 2022 Pakistan floods.

## Overview

The Copernicus GFM service calculates affected population by overlaying:
- **Flood extent data**: GFM ensemble flood maps (20m resolution)
- **Population data**: GHSL GHS-POP population density (resampled to 20m)

This reproduces the exact operational methodology used by Copernicus Emergency Management Service.

In [ ]:
import sys
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from pathlib import Path

from ds_flood_gfm.affected_population import GFMAffectedPopulationCalculator

print("✅ Imports successful")

## 1. Examine the Real GFM Flood Data

Let's start by looking at the real GFM ensemble flood extent data from Pakistan, September 15, 2022.

In [ ]:
# Load the real GFM flood data
flood_path = Path("../data/gfm/pakistan_flood_area_20220915.tif")

with rasterio.open(flood_path) as src:
    flood_data = src.read(1)
    flood_profile = src.profile
    
print("🌊 GFM Flood Data Overview:")
print(f"   Source: Copernicus GFM Ensemble")
print(f"   Date: September 15, 2022")
print(f"   Location: Pakistan")
print(f"   Shape: {flood_data.shape}")
print(f"   CRS: {src.crs}")
print(f"   Resolution: {src.res[0]:.0f}m")
print(f"   Bounds: {src.bounds}")

# Analyze flood values
unique_values = np.unique(flood_data)
print(f"\n📊 Flood Data Values:")
for value in unique_values:
    count = np.sum(flood_data == value)
    percentage = 100 * count / flood_data.size
    
    if value == 0:
        label = "No flood"
    elif value == 1:
        label = "FLOOD (ensemble agreement)"
    elif value == 255:
        label = "Background/nodata"
    else:
        label = "Other"
        
    print(f"   Value {value}: {count:,} pixels ({percentage:.1f}%) - {label}")

# Calculate actual flood statistics
flood_pixels = np.sum(flood_data == 1)
flood_area_km2 = flood_pixels * (20 * 20) / 1_000_000  # 20m pixels to km²

print(f"\n🏞️ Flood Impact Statistics:")
print(f"   Flooded pixels: {flood_pixels:,}")
print(f"   Flooded area: {flood_area_km2:.2f} km²")
print(f"   Flood percentage: {100*flood_pixels/flood_data.size:.3f}%")

## 2. Visualize the Flood Extent

Let's create a visualization of the flood data to understand the spatial distribution.

In [ ]:
# Create flood visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Full area overview
flood_display = np.where(flood_data == 1, 2,    # Flood = red
                np.where(flood_data == 0, 1,    # No flood = blue  
                        0))                     # Background = black

colors = ['black', 'lightblue', 'red']
im1 = ax1.imshow(flood_display, cmap=plt.matplotlib.colors.ListedColormap(colors))
ax1.set_title('GFM Flood Extent - Full Area\n(Red = Flood, Blue = No Flood, Black = Background)')
ax1.set_xlabel('Pixel X')
ax1.set_ylabel('Pixel Y')

# Zoomed view of flood areas only
flood_only = np.where(flood_data == 1, 1, 0)
im2 = ax2.imshow(flood_only, cmap='Reds')
ax2.set_title(f'Flood Areas Only\n{flood_pixels:,} flooded pixels ({flood_area_km2:.1f} km²)')
ax2.set_xlabel('Pixel X')
ax2.set_ylabel('Pixel Y')

plt.tight_layout()
plt.show()

print(f"📍 This flood event occurred in Pakistan during the severe 2022 monsoon flooding.")
print(f"📍 The data shows {flood_pixels:,} flooded pixels across {flood_area_km2:.1f} km².")

## 3. Examine the Real GHSL Population Data

Now let's look at the real GHSL population data that has been reprojected to match our flood area.

In [ ]:
# Load the real GHSL population data
pop_path = Path("../data/ghsl/flood_area_real_population.tif")

with rasterio.open(pop_path) as src:
    pop_data = src.read(1)
    pop_profile = src.profile

print("👥 GHSL Population Data Overview:")
print(f"   Source: GHSL GHS-POP R2023A 2020")
print(f"   Original Resolution: 3 arc-second (~100m)")
print(f"   Processed Resolution: 20m (reprojected to match flood data)")
print(f"   Shape: {pop_data.shape}")
print(f"   Total Population: {np.sum(pop_data):,.0f} people")

# Population statistics
non_zero_pixels = np.count_nonzero(pop_data)
max_density = np.max(pop_data)
mean_density = np.mean(pop_data[pop_data > 0]) if non_zero_pixels > 0 else 0

print(f"\n📊 Population Distribution:")
print(f"   Populated pixels: {non_zero_pixels:,} ({100*non_zero_pixels/pop_data.size:.1f}%)")
print(f"   Max density per pixel: {max_density:.2f} people")
print(f"   Mean density (populated areas): {mean_density:.2f} people/pixel")
print(f"   Overall density: {np.mean(pop_data):.3f} people/pixel")

## 4. Visualize Population Distribution

In [ ]:
# Create population visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Population density map
pop_display = np.where(pop_data > 0, pop_data, np.nan)  # Hide zero values
im1 = ax1.imshow(pop_display, cmap='YlOrRd', vmin=0, vmax=np.percentile(pop_data[pop_data > 0], 95))
ax1.set_title(f'GHSL Population Density\n{np.sum(pop_data):,.0f} total people')
ax1.set_xlabel('Pixel X')
ax1.set_ylabel('Pixel Y')
plt.colorbar(im1, ax=ax1, label='People per pixel')

# Binary populated areas
pop_binary = np.where(pop_data > 0, 1, 0)
im2 = ax2.imshow(pop_binary, cmap='Blues')
ax2.set_title(f'Populated Areas\n{non_zero_pixels:,} populated pixels')
ax2.set_xlabel('Pixel X')
ax2.set_ylabel('Pixel Y')

plt.tight_layout()
plt.show()

print(f"🏘️ Population is distributed across {non_zero_pixels:,} pixels in this area.")
print(f"🏘️ This represents {100*non_zero_pixels/pop_data.size:.1f}% of the total area.")

## 5. GFM Affected Population Calculation

Now we'll run the official GFM methodology to calculate affected population by overlaying flood and population data.

In [ ]:
# Initialize the GFM calculator
calculator = GFMAffectedPopulationCalculator()

print("🔄 Running GFM Affected Population Calculation...")
print("   This reproduces the exact Copernicus operational methodology.")
print()

# Run the complete calculation
results = calculator.process_gfm_affected_population(
    flood_path,
    pop_path,
    output_path=Path("../results/pakistan_affected_population_demo.tif")
)

print("\n" + "=" * 70)
print("GFM AFFECTED POPULATION RESULTS")
print("=" * 70)

## 6. Analyze the Results

Let's examine the calculated affected population and understand what the results mean.

In [ ]:
# Extract and display results
stats = results['statistics']

print("📊 FLOOD IMPACT SUMMARY:")
print(f"   Flooded Area: {stats['flooded_area_pixels']:,} pixels ({stats['flooded_area_pixels'] * 0.0004:.2f} km²)")
print(f"   Total Population in Area: {stats['total_population_in_area']:,.0f} people")
print(f"   Affected Population: {stats['total_affected_population']:,.0f} people")

if stats['total_population_in_area'] > 0:
    pct_affected = (stats['total_affected_population'] / stats['total_population_in_area']) * 100
    print(f"   Percentage Affected: {pct_affected:.4f}%")

print(f"\n🎯 SPATIAL ANALYSIS:")
print(f"   Populated pixels affected: {stats['populated_pixels_affected']:,}")
print(f"   Max people per affected pixel: {stats['max_affected_per_pixel']:.2f}")

if stats['flooded_area_pixels'] > 0:
    avg_per_flood_pixel = stats['mean_affected_per_flooded_pixel']
    print(f"   Average people per flooded pixel: {avg_per_flood_pixel:.3f}")

print(f"\n⚙️ METHODOLOGY VALIDATION:")
print(f"   ✅ Real Copernicus GFM ensemble flood data")
print(f"   ✅ Real GHSL GHS-POP 2020 population data")
print(f"   ✅ 20m spatial resolution")
print(f"   ✅ Pixel-by-pixel overlay calculation")
print(f"   ✅ Equi7Grid Asia coordinate system ({results['coordinate_system']})")

## 7. Visualize Flood-Population Overlay

Let's create a comprehensive visualization showing how flood and population data overlap.

In [ ]:
# Load the affected population raster
affected_path = Path("../results/pakistan_affected_population_demo.tif")
with rasterio.open(affected_path) as src:
    affected_data = src.read(1)

# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Flood extent
flood_display = np.where(flood_data == 1, 1, 0)
im1 = axes[0,0].imshow(flood_display, cmap='Blues')
axes[0,0].set_title(f'Flood Extent\n{np.sum(flood_display):,} flooded pixels')
axes[0,0].set_ylabel('Pixel Y')

# 2. Population density
pop_display = np.where(pop_data > 0, pop_data, np.nan)
im2 = axes[0,1].imshow(pop_display, cmap='YlOrRd', vmin=0, vmax=np.percentile(pop_data[pop_data > 0], 95))
axes[0,1].set_title(f'Population Density\n{np.sum(pop_data):,.0f} total people')
plt.colorbar(im2, ax=axes[0,1], label='People/pixel', shrink=0.8)

# 3. Overlay: flood + population
overlay = np.zeros_like(flood_data, dtype=float)
overlay = np.where((flood_data == 1) & (pop_data > 0), 3,     # Flood + Population = red
          np.where(flood_data == 1, 2,                        # Flood only = orange
          np.where(pop_data > 0, 1, 0)))                      # Population only = yellow

colors = ['white', 'yellow', 'orange', 'red']
labels = ['Neither', 'Population only', 'Flood only', 'Flood + Population']
cmap = plt.matplotlib.colors.ListedColormap(colors)

im3 = axes[1,0].imshow(overlay, cmap=cmap, vmin=0, vmax=3)
axes[1,0].set_title('Flood-Population Overlay\n(Red = Flood impacts population)')
axes[1,0].set_xlabel('Pixel X')
axes[1,0].set_ylabel('Pixel Y')

# 4. Affected population
affected_display = np.where(affected_data > 0, affected_data, np.nan)
im4 = axes[1,1].imshow(affected_display, cmap='Reds')
axes[1,1].set_title(f'Affected Population\n{np.sum(affected_data):,.0f} people affected')
axes[1,1].set_xlabel('Pixel X')
if np.sum(affected_data) > 0:
    plt.colorbar(im4, ax=axes[1,1], label='Affected people/pixel', shrink=0.8)

plt.tight_layout()
plt.show()

# Summary statistics
overlap_pixels = np.sum((flood_data == 1) & (pop_data > 0))
print(f"\n🔍 SPATIAL OVERLAP ANALYSIS:")
print(f"   Pixels with flood only: {np.sum((flood_data == 1) & (pop_data == 0)):,}")
print(f"   Pixels with population only: {np.sum((flood_data != 1) & (pop_data > 0)):,}")
print(f"   Pixels with BOTH flood and population: {overlap_pixels:,}")
print(f"   Affected population total: {np.sum(affected_data):,.1f} people")

## 8. Understanding the Results

Let's interpret what these results mean in the context of the 2022 Pakistan floods.

In [ ]:
# Calculate some interpretive statistics
total_area_km2 = flood_data.size * (20 * 20) / 1_000_000
pop_density_per_km2 = np.sum(pop_data) / total_area_km2
flood_pop_density = np.sum(affected_data) / flood_area_km2 if flood_area_km2 > 0 else 0

print("🌍 CONTEXTUAL ANALYSIS:")
print(f"   Analysis covers: {total_area_km2:.0f} km² in Pakistan")
print(f"   Population density: {pop_density_per_km2:.1f} people/km²")
print(f"   Flood covered: {flood_area_km2:.1f} km² ({100*flood_area_km2/total_area_km2:.2f}% of area)")
print(f"   Population density in flooded areas: {flood_pop_density:.1f} people/km²")

print(f"\n📈 INTERPRETATION:")
if stats['total_affected_population'] == 0:
    print(f"   • This flood event occurred in sparsely populated areas")
    print(f"   • Only {overlap_pixels:,} pixels had both flood and population")
    print(f"   • Demonstrates that not all floods affect populated areas")
    print(f"   • The methodology correctly identifies minimal population impact")
elif stats['total_affected_population'] > 1000:
    print(f"   • Significant flood impact: {stats['total_affected_population']:,.0f} people affected")
    print(f"   • {pct_affected:.2f}% of area population impacted")
    print(f"   • Flood occurred in populated regions")
else:
    print(f"   • Moderate flood impact: {stats['total_affected_population']:,.0f} people affected")
    print(f"   • Limited overlap between flood extent and populated areas")

print(f"\n✅ METHODOLOGY SUCCESS:")
print(f"   • Successfully reproduced Copernicus GFM operational methodology")
print(f"   • Used real satellite flood data and real population census data")
print(f"   • Calculated precise pixel-level population impact")
print(f"   • Results can be used for emergency response and planning")

## 9. Export Results

The affected population raster has been saved and can be used for further analysis or integration with GIS systems.

In [ ]:
print("💾 OUTPUT FILES CREATED:")
print(f"   Affected population raster: {affected_path}")
print(f"   File size: {affected_path.stat().st_size / 1024:.1f} KB")
print(f"   Format: GeoTIFF")
print(f"   Coordinate system: {results['coordinate_system']}")
print(f"   Resolution: {results['resolution']}")

print(f"\n📊 SUMMARY STATISTICS:")
print(f"   Analysis date: September 15, 2022")
print(f"   Location: Pakistan flood area")
print(f"   Flood extent: {stats['flooded_area_pixels']:,} pixels ({flood_area_km2:.1f} km²)")
print(f"   Total population: {stats['total_population_in_area']:,.0f} people")
print(f"   Affected population: {stats['total_affected_population']:,.0f} people")
print(f"   Methodology: GFM ensemble majority voting + GHSL overlay")

print(f"\n🎯 This analysis demonstrates the complete GFM affected population workflow")
print(f"using real operational data from the Copernicus Emergency Management Service.")